# Cloud performance variability prediction#

In [24]:
import csv
import pandas as pd
import numpy as np
import string

In [2]:
import os
dict = {}

source_dir = './cloudlab-benchmarks-osdi18-data-and-nodebooks/data/raw-data/'
for f in os.listdir(source_dir):
    if ".csv" in f:
        dict[os.path.splitext(f)[0]] = pd.read_csv(source_dir + f)

In [3]:
dict['disk_info'].T

,0,1,2,3,4,5,6,7,8,9,...,17699,17700,17701,17702,17703,17704,17705,17706,17707,17708
run_uuid,d29989cd-c414-4af5-b04f-f29561b0240d,a3d37542-7bf9-468f-a630-700715c9b1ff,a3d37542-7bf9-468f-a630-700715c9b1ff,368d9894-c37f-49de-a073-3cab47f69a3a,368d9894-c37f-49de-a073-3cab47f69a3a,9fbcd3b2-c23e-4e6c-a19b-54344b48890b,9fbcd3b2-c23e-4e6c-a19b-54344b48890b,9fbcd3b2-c23e-4e6c-a19b-54344b48890b,b8753918-c049-4bf7-966e-d16b26e6de12,b8753918-c049-4bf7-966e-d16b26e6de12,...,fc1aaa02-de9f-43ec-99cf-1c548904b435,fc1aaa02-de9f-43ec-99cf-1c548904b435,d0932a2b-914d-4461-adf1-b46d4c01ccc9,d0932a2b-914d-4461-adf1-b46d4c01ccc9,d0932a2b-914d-4461-adf1-b46d4c01ccc9,db427808-47f3-4599-9a24-3973c4425788,d2731139-52e3-4697-a0bc-a8a6beab0255,b1389312-a648-4e87-9046-085644802b02,b23f3655-2293-4be7-8cd0-be41ab0357be,faf52960-818e-48e1-88d0-2bf58bfa0771
timestamp,1495271912,1495272018,1495272018,1495272078,1495272078,1495272065,1495272065,1495272065,1495272056,1495272056,...,1522852581,1522852581,1522852581,1522852581,1522852581,1522865516,1522865516,1522865512,1522865513,1522865528
nodeid,ms0225,clnode123,clnode123,clnode102,clnode102,c220g2-010817,c220g2-010817,c220g2-010817,c220g1-030808,c220g1-030808,...,c220g2-011301,c220g2-011301,c220g2-011320,c220g2-011320,c220g2-011320,ms0134,ms0107,ms0839,ms0802,ms0238
nodeuuid,68285417-72da-11e4-85c8-179a28bd1b8c,cedf924b-fee2-11e5-ba07-000099989701,cedf924b-fee2-11e5-ba07-000099989701,f2537c2f-fee1-11e5-ba07-000099989701,f2537c2f-fee1-11e5-ba07-000099989701,21b9599e-e772-11e5-a488-a3bb994732a6,21b9599e-e772-11e5-a488-a3bb994732a6,21b9599e-e772-11e5-a488-a3bb994732a6,fcfbab77-2931-11e6-8934-779211246dcb,fcfbab77-2931-11e6-8934-779211246dcb,...,1c88d71c-e8ec-11e5-a488-a3bb994732a6,1c88d71c-e8ec-11e5-a488-a3bb994732a6,1a13de4c-e8ec-11e5-a488-a3bb994732a6,1a13de4c-e8ec-11e5-a488-a3bb994732a6,1a13de4c-e8ec-11e5-a488-a3bb994732a6,466b2b8c-72d5-11e4-85c8-179a28bd1b8c,43cc585a-72d5-11e4-85c8-179a28bd1b8c,cd9b0d15-8fbf-11e6-8c08-a53a3a50acf8,e218b7a2-8f4b-11e6-8c08-a53a3a50acf8,66ea8663-72da-11e4-85c8-179a28bd1b8c
disk_name,/dev/sda,/dev/sdb,/dev/sda,/dev/sdb,/dev/sda,/dev/sdb,/dev/sdc,/dev/sda,/dev/sdb,/dev/sdc,...,/dev/sdc,/dev/sda,/dev/sdb,/dev/sdc,/dev/sda,/dev/sda,/dev/sda,/dev/nvme0n1,/dev/nvme0n1,/dev/sda
disk_model,XR0120GEBLT,ST91000640NS,ST91000640NS,ST91000640NS,ST91000640NS,HUC101212CSS600,INTEL SSDSC2BB48,HUC101212CSS600,ST1200MM0007,INTEL SSDSC2BB48,...,INTEL SSDSC2BB48,HUC101212CSS600,HUC101212CSS600,INTEL SSDSC2BB48,HUC101212CSS600,XR0120GEBLT,XR0120GEBLT,THNSN5256GPU7 TOSHIBA,THNSN5256GPU7 TOSHIBA,XR0120GEBLT
disk_serial,14230C9CD4C4,9XG9FW2C,9XG9MB8Q,9XG9MA8Z,9XG9L6T1,5000cca072a93574,BTWL5235053V480QGN,5000cca072aa1bc4,5000c5007de1e3af,CVWL4442017V480QGN,...,PHWL538201SN480QGN,5000cca072a924f0,5000cca072aa2014,PHWL538201JS480QGN,5000cca0729af008,14230C9CDEEE,14230C9CDED0,563S1002TUFV,563S1021TUFV,14230C9CDA59
disk_size,111.8G,931.5G,931.5G,931.5G,931.5G,1.1T,447.1G,1.1T,1.1T,447.1G,...,447.1G,1.1T,1.1T,447.1G,1.1T,111.8G,111.8G,238.5G,238.5G,111.8G
npartitions,3,0,4,0,4,0,0,4,0,0,...,0,4,0,0,4,3,3,4,4,3
disk_type,SSD,HDD,HDD,HDD,HDD,HDD,SSD,HDD,HDD,SSD,...,SSD,HDD,HDD,SSD,HDD,SSD,SSD,SSD,SSD,SSD


In [4]:
dict['disk_results'].T

,0,1,2,3,4,5,6,7,8,9,...,141398,141399,141400,141401,141402,141403,141404,141405,141406,141407
run_uuid,d29989cd-c414-4af5-b04f-f29561b0240d,d29989cd-c414-4af5-b04f-f29561b0240d,d29989cd-c414-4af5-b04f-f29561b0240d,d29989cd-c414-4af5-b04f-f29561b0240d,d29989cd-c414-4af5-b04f-f29561b0240d,d29989cd-c414-4af5-b04f-f29561b0240d,d29989cd-c414-4af5-b04f-f29561b0240d,d29989cd-c414-4af5-b04f-f29561b0240d,a3d37542-7bf9-468f-a630-700715c9b1ff,a3d37542-7bf9-468f-a630-700715c9b1ff,...,b23f3655-2293-4be7-8cd0-be41ab0357be,b23f3655-2293-4be7-8cd0-be41ab0357be,faf52960-818e-48e1-88d0-2bf58bfa0771,faf52960-818e-48e1-88d0-2bf58bfa0771,faf52960-818e-48e1-88d0-2bf58bfa0771,faf52960-818e-48e1-88d0-2bf58bfa0771,faf52960-818e-48e1-88d0-2bf58bfa0771,faf52960-818e-48e1-88d0-2bf58bfa0771,faf52960-818e-48e1-88d0-2bf58bfa0771,faf52960-818e-48e1-88d0-2bf58bfa0771
timestamp,1495271912,1495271912,1495271912,1495271912,1495271912,1495271912,1495271912,1495271912,1495272018,1495272018,...,1522865513,1522865513,1522865528,1522865528,1522865528,1522865528,1522865528,1522865528,1522865528,1522865528
nodeid,ms0225,ms0225,ms0225,ms0225,ms0225,ms0225,ms0225,ms0225,clnode123,clnode123,...,ms0802,ms0802,ms0238,ms0238,ms0238,ms0238,ms0238,ms0238,ms0238,ms0238
nodeuuid,68285417-72da-11e4-85c8-179a28bd1b8c,68285417-72da-11e4-85c8-179a28bd1b8c,68285417-72da-11e4-85c8-179a28bd1b8c,68285417-72da-11e4-85c8-179a28bd1b8c,68285417-72da-11e4-85c8-179a28bd1b8c,68285417-72da-11e4-85c8-179a28bd1b8c,68285417-72da-11e4-85c8-179a28bd1b8c,68285417-72da-11e4-85c8-179a28bd1b8c,cedf924b-fee2-11e5-ba07-000099989701,cedf924b-fee2-11e5-ba07-000099989701,...,e218b7a2-8f4b-11e6-8c08-a53a3a50acf8,e218b7a2-8f4b-11e6-8c08-a53a3a50acf8,66ea8663-72da-11e4-85c8-179a28bd1b8c,66ea8663-72da-11e4-85c8-179a28bd1b8c,66ea8663-72da-11e4-85c8-179a28bd1b8c,66ea8663-72da-11e4-85c8-179a28bd1b8c,66ea8663-72da-11e4-85c8-179a28bd1b8c,66ea8663-72da-11e4-85c8-179a28bd1b8c,66ea8663-72da-11e4-85c8-179a28bd1b8c,66ea8663-72da-11e4-85c8-179a28bd1b8c
testname,randwrite,randread,randwrite,write,randread,read,read,write,randread,read,...,read,randwrite,randwrite,randread,randwrite,write,randread,read,read,write
runtime,720.002,105.565,720.283,720.002,140.652,56.591,138.604,88.733,720.006,91.037,...,63.39,15.824,720.003,105.962,720.317,720.001,140.03,55.047,138.682,87.561
size,1.4021e+06,1.04858e+07,1.41308e+06,1.40205e+06,1.04858e+07,1.04858e+07,1.04858e+07,1.04858e+07,436284,1.04858e+07,...,1.04858e+07,1.04858e+07,1.41186e+06,1.04858e+07,1.42291e+06,1.41267e+06,1.04858e+07,1.04858e+07,1.04858e+07,1.04858e+07
max,2024,172912,2023,2011,102976,190536,77352,122304,752,121544,...,176960,776616,2000,176656,2015,2000,103976,195185,77088,122059
min,1832,7,0,1896,25928,180096,73408,109768,262,95235,...,138608,460688,1780,7,0,1896,25672,182440,72904,112896
mean,1949.05,98894.2,1961.17,1948.92,74530,185430,75659.6,118267,606.078,115254,...,165450,661000,1962.66,98654.2,1975.46,1963.71,74884.6,190613,75617.2,119856


In [5]:
# dict['env_info']

In [6]:
# dict['fio_info'].T

In [7]:
# dict['iperf3_info'].T

In [8]:
# dict['iperf3_results'].T

In [9]:
# dict['mem_results'].T

In [10]:
# dict['membench_info'].T

In [11]:
# dict['network_info'].T

In [12]:
# dict['ping_info'].T

In [13]:
# dict['ping_results'].T

In [14]:
# dict['stream_info'].T

In [28]:
disk = pd.merge(dict['disk_results'], dict['env_info'], 
                on=['run_uuid','nodeid','nodeuuid','timestamp'])
disk['disk_name'] = disk['device'].apply(lambda x: x.rstrip(string.digits) if 'nvm' not in x else x[:-2])
disk['disk_name'].unique()
disk = pd.merge(disk, dict['disk_info'], 
                on=['run_uuid','timestamp','nodeuuid','disk_name','nodeid'])

In [29]:
disk.T

,0,1,2,3,4,5,6,7,8,9,...,141398,141399,141400,141401,141402,141403,141404,141405,141406,141407
run_uuid,d29989cd-c414-4af5-b04f-f29561b0240d,d29989cd-c414-4af5-b04f-f29561b0240d,d29989cd-c414-4af5-b04f-f29561b0240d,d29989cd-c414-4af5-b04f-f29561b0240d,d29989cd-c414-4af5-b04f-f29561b0240d,d29989cd-c414-4af5-b04f-f29561b0240d,d29989cd-c414-4af5-b04f-f29561b0240d,d29989cd-c414-4af5-b04f-f29561b0240d,a3d37542-7bf9-468f-a630-700715c9b1ff,a3d37542-7bf9-468f-a630-700715c9b1ff,...,b23f3655-2293-4be7-8cd0-be41ab0357be,b23f3655-2293-4be7-8cd0-be41ab0357be,faf52960-818e-48e1-88d0-2bf58bfa0771,faf52960-818e-48e1-88d0-2bf58bfa0771,faf52960-818e-48e1-88d0-2bf58bfa0771,faf52960-818e-48e1-88d0-2bf58bfa0771,faf52960-818e-48e1-88d0-2bf58bfa0771,faf52960-818e-48e1-88d0-2bf58bfa0771,faf52960-818e-48e1-88d0-2bf58bfa0771,faf52960-818e-48e1-88d0-2bf58bfa0771
timestamp,1495271912,1495271912,1495271912,1495271912,1495271912,1495271912,1495271912,1495271912,1495272018,1495272018,...,1522865513,1522865513,1522865528,1522865528,1522865528,1522865528,1522865528,1522865528,1522865528,1522865528
nodeid,ms0225,ms0225,ms0225,ms0225,ms0225,ms0225,ms0225,ms0225,clnode123,clnode123,...,ms0802,ms0802,ms0238,ms0238,ms0238,ms0238,ms0238,ms0238,ms0238,ms0238
nodeuuid,68285417-72da-11e4-85c8-179a28bd1b8c,68285417-72da-11e4-85c8-179a28bd1b8c,68285417-72da-11e4-85c8-179a28bd1b8c,68285417-72da-11e4-85c8-179a28bd1b8c,68285417-72da-11e4-85c8-179a28bd1b8c,68285417-72da-11e4-85c8-179a28bd1b8c,68285417-72da-11e4-85c8-179a28bd1b8c,68285417-72da-11e4-85c8-179a28bd1b8c,cedf924b-fee2-11e5-ba07-000099989701,cedf924b-fee2-11e5-ba07-000099989701,...,e218b7a2-8f4b-11e6-8c08-a53a3a50acf8,e218b7a2-8f4b-11e6-8c08-a53a3a50acf8,66ea8663-72da-11e4-85c8-179a28bd1b8c,66ea8663-72da-11e4-85c8-179a28bd1b8c,66ea8663-72da-11e4-85c8-179a28bd1b8c,66ea8663-72da-11e4-85c8-179a28bd1b8c,66ea8663-72da-11e4-85c8-179a28bd1b8c,66ea8663-72da-11e4-85c8-179a28bd1b8c,66ea8663-72da-11e4-85c8-179a28bd1b8c,66ea8663-72da-11e4-85c8-179a28bd1b8c
testname,randwrite,randread,randwrite,write,randread,read,read,write,randread,read,...,read,randwrite,randwrite,randread,randwrite,write,randread,read,read,write
runtime,720.002,105.565,720.283,720.002,140.652,56.591,138.604,88.733,720.006,91.037,...,63.39,15.824,720.003,105.962,720.317,720.001,140.03,55.047,138.682,87.561
size,1.4021e+06,1.04858e+07,1.41308e+06,1.40205e+06,1.04858e+07,1.04858e+07,1.04858e+07,1.04858e+07,436284,1.04858e+07,...,1.04858e+07,1.04858e+07,1.41186e+06,1.04858e+07,1.42291e+06,1.41267e+06,1.04858e+07,1.04858e+07,1.04858e+07,1.04858e+07
max,2024,172912,2023,2011,102976,190536,77352,122304,752,121544,...,176960,776616,2000,176656,2015,2000,103976,195185,77088,122059
min,1832,7,0,1896,25928,180096,73408,109768,262,95235,...,138608,460688,1780,7,0,1896,25672,182440,72904,112896
mean,1949.05,98894.2,1961.17,1948.92,74530,185430,75659.6,118267,606.078,115254,...,165450,661000,1962.66,98654.2,1975.46,1963.71,74884.6,190613,75617.2,119856
